In [1]:
# Cell 1. Импорты и константы
import pandas as pd
import numpy as np
from datetime import timedelta

# параметры дискретизации
BINNING_SPEC = {
    'Total Fwd Packets':      ('uniform', 5),
    'Total Backward Packets': ('uniform', 5),
    'Total Length of Fwd Packets': ('uniform', 5),
    'Total Length of Bwd Packets': ('uniform', 5),
    'Active Mean':            ('uniform', 5),
    'Idle Mean':              ('uniform', 5),
    'Flow Duration':          ('uniform', 5)
}

SELECTED_COLS = [
    'Total Fwd Packets','Total Backward Packets',
    'Total Length of Fwd Packets','Total Length of Bwd Packets',
    'Active Mean','Idle Mean','Flow Duration',
    'Destination IP','Destination Port',
    'FIN Flag Count','SYN Flag Count',
    'RST Flag Count','PSH Flag Count','ACK Flag Count'
]

FEATURE_GROUPS = {
    'network': [
        'Destination IP','Destination Port',
        'FIN Flag Count','SYN Flag Count',
        'RST Flag Count','PSH Flag Count','ACK Flag Count'
    ],
    'temporal': ['Flow Duration'],
    'traffic': [
        'Total Fwd Packets','Total Backward Packets',
        'Total Length of Fwd Packets','Total Length of Bwd Packets',
        'Active Mean','Idle Mean'
    ]
}

# Порог срабатывания: сколько отфильтрованных правил нужно «пробить»
MIN_RULES = 150


In [2]:
# Cell 2. Функции дискретизации и извлечения фич для одной строки

def discretize_columns(df_in, spec, cols):
    df2 = df_in[['Timestamp'] + cols].copy()
    for col, (method, bins) in spec.items():
        if col not in cols: 
            continue
        arr = df2[col].astype(float)
        labels = [f"bin{i}" for i in range(bins)]
        if method == 'quantile':
            df2[col] = pd.qcut(arr, q=bins, labels=labels, duplicates='drop').astype(str)
        else:
            df2[col] = pd.cut(arr, bins=bins, labels=labels).astype(str)
    return df2

def row_features(raw, disc):
    """
    Собирает все SELECTED_COLS в формат 'Feature=val' для одной строки.
    """
    feats = set()
    for col in SELECTED_COLS:
        if col in BINNING_SPEC:
            feats.add(f"{col}={disc[col]}")
        else:
            feats.add(f"{col}={raw[col]}")
    return feats


In [3]:
# Cell 3. Загрузка и дискретизация датасетов

# 3.1 размеченный (для оценки)
df_labeled = pd.read_csv(
    r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\diplom-project\diplom\test\result\Date\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_sampled_v3.csv',
    parse_dates=['Timestamp']
)

# 3.2 неразмеченный (для реального теста, но здесь не нужен)
df_unlabeled = pd.read_csv(
    r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\diplom-project\diplom\test\result\Date\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_sampled_v4.csv',
    parse_dates=['Timestamp']
)

# 3.3 дискретизируем оба
df_disc_labeled   = discretize_columns(df_labeled,   BINNING_SPEC, SELECTED_COLS)
df_disc_unlabeled = discretize_columns(df_unlabeled, BINNING_SPEC, SELECTED_COLS)


In [4]:
# Cell 4. Загрузка Top-K-шаблона и сбор DDoS-фич

tmpl_df = pd.read_csv('generated_attack_templates.csv')  # поля: Attack, Group, Values
ddos_template = set()

for _, row in tmpl_df[tmpl_df['Attack']=='DDOS'].iterrows():
    # 'Values' может быть "Flow Duration=bin0|bin3" или "Destination IP=192.168.10.50"
    col, bins = row['Values'].split('=',1)
    for b in bins.split('|'):
        ddos_template.add(f"{col}={b}")


In [5]:
# ─── Cell 5. Загрузка неразмеченных правил и фильтрация по шаблону ─────────────



# читаем AR-правила без меток
rules_df = pd.read_csv('association_rules_no_label.csv')  # Antecedent, Consequent, Support, Confidence, Lift
raw_rules = []
for _, r in rules_df.iterrows():
    A = set(x.strip() for x in r['Antecedent'].strip('{}').split(','))
    B = r['Consequent']
    sup  = r['Support']
    conf = r['Confidence']
    lift = r['Lift']
    raw_rules.append((A, B, sup, conf, lift))

# теперь фильтруем по шаблону и по длине A ≥ 2
filtered_rules = [
    (A, B)
    for (A, B, sup, conf, lift) in raw_rules
    # правило должно состоять минимум из 2 признаков
    if len(A) >= 4
    # все его фичи лежат в DDoS-шаблоне
    and A.issubset(ddos_template)
    # и сам consequent должен быть в шаблоне
    and (B in ddos_template)
]

print(f"Всего raw_rules: {len(raw_rules)}, после filter len(A)>=2 и по шаблону: {len(filtered_rules)}")


Всего raw_rules: 1293, после filter len(A)>=2 и по шаблону: 419


In [6]:
# Cell 6. Детекция по строкам и оценка качества

preds = []
TP = FP = TN = FN = 0

for idx, raw in df_labeled.iterrows():
    disc  = df_disc_labeled.loc[idx]
    feats = row_features(raw, disc)

    # обязательная проверка: хотя бы по одному фичу из каждой группы
    ok = True
    for grp, cols in FEATURE_GROUPS.items():
        # берем из шаблона только фичи этой группы
        grp_feats = {f for f in ddos_template if any(f.startswith(c+"=") for c in cols)}
        if not (grp_feats & feats):
            ok = False
            break

    if not ok:
        pred = 'BENIGN'
    else:
        # считаем, сколько отфильтрованных правил ≡ A⇒B 'прошли'
        fires = sum(1 for A,B in filtered_rules if A.issubset(feats) and (B in feats))
        pred  = 'DDoS' if fires >= MIN_RULES else 'BENIGN'

    true = raw['Label']
    if pred == true:
        if pred == 'DDoS': TP += 1
        else:             TN += 1
    else:
        if pred == 'DDoS': FP += 1
        else:             FN += 1

    rec = raw.to_dict()
    rec['Predicted'] = pred
    preds.append(rec)

precision = TP / (TP + FP) if TP + FP else 0
recall    = TP / (TP + FN) if TP + FN else 0
print(f"Row‐level Precision: {precision:.2f}, Recall: {recall:.2f}")
print(f"TP={TP}, FP={FP}, TN={TN}, FN={FN}")

# сохраняем все строки с предсказаниями
pd.DataFrame(preds).to_csv('row_level_predictions.csv', index=False)
print("Готово: row_level_predictions.csv")


Row‐level Precision: 0.89, Recall: 0.83
TP=24907, FP=3084, TN=66916, FN=5093
Готово: row_level_predictions.csv


In [7]:
# ─── Cell X+: Grid Search по min_len и min_rules ─────────────────────────────
import pandas as pd
from itertools import product

# Диапазоны, которые хотим перебрать
min_len_list   = [2, 3, 4, 5]      # минимальная длина Antecedent
min_rules_list = [50, 100, 150, 200, 250]  # пороги для fires >= min_rules

results = []

for min_len, min_rules in product(min_len_list, min_rules_list):
    # 1) Отбираем правила по длине A >= min_len
    filtered = [
        (A, B)
        for (A, B, sup, conf, lift) in raw_rules
        if len(A) >= min_len
           and A.issubset(ddos_template)
           and (B in ddos_template)
    ]

    # 2) Считаем метрики на уровне строк
    TP = FP = TN = FN = 0
    for idx, raw in df_labeled.iterrows():
        disc  = df_disc_labeled.loc[idx]
        feats = row_features(raw, disc)

        # групповой фильтр: хотя бы одна фича из каждой группы
        ok = all(
            (set(f for f in ddos_template if any(f.startswith(c+"=") for c in cols)) & feats)
            for grp, cols in FEATURE_GROUPS.items()
        )
        if not ok:
            pred = 'BENIGN'
        else:
            fires = sum(1 for A, B in filtered if A.issubset(feats) and (B in feats))
            pred  = 'DDoS' if fires >= min_rules else 'BENIGN'

        true = raw['Label']
        if pred == true:
            if pred == 'DDoS': TP += 1
            else:             TN += 1
        else:
            if pred == 'DDoS': FP += 1
            else:             FN += 1

    precision = TP / (TP + FP) if TP + FP else 0
    recall    = TP / (TP + FN) if TP + FN else 0
    f1        = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    results.append({
        'min_len': min_len,
        'min_rules': min_rules,
        'num_rules': len(filtered),
        'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN,
        'precision': precision,
        'recall': recall,
        'f1': f1
    })

# 3) Собираем и сортируем
df_res = pd.DataFrame(results)
df_res = df_res.sort_values(['f1','precision'], ascending=False).reset_index(drop=True)

print(df_res)

# Сохраняем в CSV для дальнейшего анализа
df_res.to_csv('gridsearch_len_rules.csv', index=False)
print("Saved → gridsearch_len_rules.csv")


    min_len  min_rules  num_rules     TP     FP     TN     FN  precision  \
0         4        150        419  24907   3084  66916   5093   0.889822   
1         3        200        682  24907  15227  54773   5093   0.620596   
2         4        100        419  24907  15227  54773   5093   0.620596   
3         2        200        771  24907  15467  54533   5093   0.616907   
4         3        150        682  24907  16074  53926   5093   0.607769   
5         2        150        771  24907  17107  52893   5093   0.592826   
6         4        200        419  13636    427  69573  16364   0.969637   
7         4         50        419  29558  39072  30928    442   0.430686   
8         3         50        682  29952  40545  29455     48   0.424869   
9         2         50        771  29952  40555  29445     48   0.424809   
10        3        250        682  13636   3854  66146  16364   0.779646   
11        2        100        771  25878  39077  30923   4122   0.398399   
12        3 

In [8]:
# ─── Cell Z: Диагностика причин FP/FN ───────────────────────────────────────
import pandas as pd

# ← Подставь сюда те же константы, что в своём detection-скрипте
MIN_RULES     = 150    # сколько правил должно «пробиться»
RULE_MIN_LEN  = 4    # отбрасываем слишком короткие A (len(A) >= RULE_MIN_LEN)

# 1) Подгружаем предсказания
df_preds = pd.read_csv('row_level_predictions.csv', parse_dates=['Timestamp'])

# 2) Загружаем «сырые» AR-правила (без меток), которые ты фильтруешь в коде
rules_df = pd.read_csv('association_rules_no_label_short.csv')  # или свой файл
raw_rules = []
for _, r in rules_df.iterrows():
    A = set(x.strip() for x in r['Antecedent'].strip('{}').split(','))
    B = r['Consequent']  # здесь у тебя просто 'DDOS' или 'BENIGN'
    raw_rules.append((A, B))

# 3) Отфильтровываем их ровно так же, как ты это делаешь в detect_attack 
#    (len(A)>=RULE_MIN_LEN, A⊆ddos_template и B∈ddos_template)
filtered_rules = [
    (A, B)
    for A, B in raw_rules
    if len(A) >= RULE_MIN_LEN
       and A.issubset(ddos_template)
       and (B in ddos_template)
]

# 4) Раскладываем ddos_template по группам, чтобы проверять покрытие network/temporal/traffic
ddos_template_groups = {
    grp: {f for f in ddos_template if any(f.startswith(col + '=') for col in cols)}
    for grp, cols in FEATURE_GROUPS.items()
}

# 5) Собираем diagnostics
records = []
for idx, row in df_preds.iterrows():
    true = row['Label']
    pred = row['Predicted']
    # только ошибки
    if true == pred:
        continue

    # 5.1) Фичи из исходных и дискретных
    raw  = df_labeled.loc[idx]
    disc = df_disc_labeled.loc[idx]
    feats = row_features(raw, disc)

    # 5.2) Какие группы шаблона вообще не покрылись?
    missing = [
        grp for grp, tmpl_feats in ddos_template_groups.items()
        if not (tmpl_feats & feats)
    ]

    # 5.3) Какие filtered_rules пробились?
    fired = [A for A, B in filtered_rules if A.issubset(feats)]
    n_fired = len(fired)

    # 5.4) Формируем причину
    if missing:
        reason = f"Missing template groups: {missing}"
    elif n_fired < MIN_RULES:
        reason = f"Only {n_fired} rules fired (<{MIN_RULES})"
    else:
        reason = "Passed all checks but still misclassified"

    records.append({
        'row_index':        idx,
        'true_label':       true,
        'predicted':        pred,
        'missing_groups':   missing,
        'num_rules_fired':  n_fired,
        'fired_antecedents': fired,
        'reason':           reason
    })

# 6) Сохраняем
df_diag = pd.DataFrame(records)
df_diag.to_csv('misclassification_reasons.csv', index=False)
print("✓ Saved misclassification_reasons.csv — разбор каждого FP/FN")


✓ Saved misclassification_reasons.csv — разбор каждого FP/FN
